In [ ]:
# concurrent.futures模块的主要特色是ThreadPoolExecutor和ProcessPoolExecutor类，这两个类能分别在不同的线程或进程中
# 执行可调用的对象。都在内部维护着一个工作线程或进程池，以及要执行的任务队列。


import os
import time
import sys

from concurrent import futures

import requests  # 用于发起http

POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()  # 待下载国家

BASE_URL = 'http://flupy.org/data/flags'  # 服务方

DEST_DIR = 'downloads/'  # 本地目录

MAX_WORKERS = 20  # 最多并发数

def save_flag(img, filename):  # 保存获得的字节序列到文件
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp.write(img)

def get_flag(cc):  # 发起请求获得响应
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = requests.get(url)
    return resp.content

def show(text):  # 显示下载内容
    print(text, ' retrieved.')
    #print(text, end=' ') # 不用换行，让结果显示在一行上
    #sys.stdout.flush() # 没有换行符需要显式刷新

def main(download_many):  # 调用入口，记录所需时间
    t0 = time.time()
    count = download_many(POP20_CC)
    elapsed = time.time() - t0
    msg = '\n{} flags downloaded in {:.2f}s'
    print(msg.format(count, elapsed))

def download_one(cc):  # 下载一个
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc

def download_many(cc_list): # 下载多个
    workers = min(MAX_WORKERS, len(cc_list))  # 计算实际所需并发数
    # 用并发数实例化，executor.__exit__方法会调用shutdown(wait=True)进行阻塞
    with futures.ThreadPoolExecutor(workers) as executor:         
        res = executor.map(download_one, sorted(cc_list))  # executor.map与内置map类似，但是在多个线程中并发调用download_one

    return len(list(res))  # 返回结果

main(download_many)
print('-------------')

# 模拟executor.map的行为：
def download_many(cc_list):
    cc_list = cc_list[:5]  # 只用5个作为演示
    with futures.ThreadPoolExecutor(max_workers=3) as executor:  # 临时用3作为演示
        to_do = []
        for cc in sorted(cc_list):
            future = executor.submit(download_one, cc)  # submit方法排定可调用对象的执行时间，返回一个期物future
            to_do.append(future)  # 存储期物待使用
            msg = 'Scheduled for {}: {}'
            print(msg.format(cc, future))  # 显示国家代码和对应的期物

        results = []
        for future in futures.as_completed(to_do):  # as_completed方法在期物运行结束后产出期物
            res = future.result()  # 获得期物的结。已调用过as_completed，所以不会阻塞。
            msg = '{} result: {!r}'
            print(msg.format(future, res)) # 显示期物及结果
            results.append(res)

    return len(results)

main(download_many)

# multiprocessing 库更适合CPU计算密集的并发
